## Euclidean Distance (유클리드 거리) 기반 유사도 측정

In [1]:
import os
import pymysql
import time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
import json

In [2]:
# 훈련 데이터를 데이터프레임으로 로드
train_data = pd.read_csv('../train_data/Data3_train_tag.csv', encoding='utf-8')# 데이터 파일 경로
train_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,tag1,tag2,tag3
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,NaN,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월'...",b'\x0e\x11u<\xdeM1=\xa1kX=\xd11\x06\xbe?\x03\x...,사람,슬픔,사랑
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"['첫사랑', '나선', '여자', '첫사랑', '남자', '티격태격', '로맨틱'...",b'\xddC\xf5=I@Z=%\xfd\x8b\xbc\xd8\x03==\xd6q\x...,오페라,사랑,NaN
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"['시간', '조금', '수', '있다', '적', '동네', '해수', '오누이'...","b""S\xa2\xa0<\xeb\xad\xd5<\xd6\x93\xbe\xbd\x01\...",오페라,NaN,NaN
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"['인기', '그룹', '핫바', '멤버', '가수', '희다', '은', '날',...",b'^\xb5D>\xb63\xad=P\x8e >.\x97\x02\xbe\xee+\x...,오페라,NaN,NaN
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"['어머니', '유언', '위해', '궁', '생각', '시가', '장금', '것'...",b'@\x8a\xa8=\x8e\xd7\xe0=\x1e\x10\x9b=N%\xf6\x...,오페라,사랑,아이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20882,152667,유앤잇 대구,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20191130 20191130,2019-11-30,2019-11-01,대구북구 어울아트센터 대공연장 함지홀,NaN,8세이상,80분,NaN,NaN,,"['', '']",b'\xd5q4>\x9b\x12h\xbd*\xf4\x99>\xc0\xe2`>\xe1...,NaN,NaN,NaN
20883,152711,크리스마스 이브 파티,httpticketimageinterparkcomPlayDictionaryDATAP...,뮤지컬,20191223 20191224,2019-12-23,2019-12-01,코엑스 신한카드 아티움,NaN,8세이상,120분,NaN,NaN,,"['', '']",b'\xd5q4>\x9b\x12h\xbd*\xf4\x99>\xc0\xe2`>\xe1...,NaN,NaN,NaN
20884,153002,스냅 안동,httpticketimageinterparkcomPlayDictionaryDATAP...,퍼포먼스,20191227 20191228,2019-12-27,2019-12-01,안동문화예술의전당 웅부홀,NaN,48개월이상,70분,NaN,NaN,,"['', '']",b'\xd5q4>\x9b\x12h\xbd*\xf4\x99>\xc0\xe2`>\xe1...,NaN,NaN,NaN
20885,153063,어린왕자의 크리스마스 파티 부산,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20191221 20191229,2019-12-21,2019-12-01,성원아트홀,NaN,24개월이상,50분,NaN,NaN,,"['', '']",b'\xd5q4>\x9b\x12h\xbd*\xf4\x99>\xc0\xe2`>\xe1...,NaN,NaN,NaN


In [3]:
# 현재 상영중 데이터를 데이터프레임으로 로드
present_data = pd.read_csv('../present_data/Data3_present_tag.csv', encoding='utf-8')# 데이터 파일 경로
present_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,tag1,tag2,tag3
0,190722,레미제라블,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231130 20240310,2023-11-30,2023-11-01,블루스퀘어 신한카드홀,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,NaN,180분 인터미션 20분,Les Misérables\n뮤지컬 레미제라블\n시대를 뛰어넘은\n위대한 명작\n월...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"['사랑', '용기', '희망', '대한', '서사시', '기구', '운명', '청...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...,사랑,친구,NaN
1,190036,레베카,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230819 20231119,2023-08-19,2023-08-01,블루스퀘어 신한카드홀,류정한민영기에녹테이신영숙옥주현리사장은아김보경이지혜이지수웬디윤석원임정모윤사봉김지선이은...,8세이상,170분 인터미션 20분,더 이상 수식어가 필요 없는 가장 완벽한 뮤지컬역사적 10주년 기념공연으로 돌아온 ...,불의의 사고로 아내 레베카를 잃고 힘든 나날을 보내고 있는 막심 드 윈터그는 몬테카...,불의 사고 아내 레베카 힘드다 나날 있다 막심 드 윈터 그 몬테카를로 여행 중 사랑...,"['불의', '사고', '아내', '레베카', '힘드다', '나날', '있다', '...",b'\xd8M\xd1\xbb[\x8d\x14<\x18\x1f\xf09v\x01\x0...,사람,사랑,친구
2,183052,오페라의 유령,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230721 20231119,2023-07-21,2023-07-01,샤롯데씨어터,조승우최재림김주택전동석손지수송은혜송원근황건하윤영석이상준이지영한보라박회림김아선,초등학생이상 관람가,150분 인터미션 20분,13년 간의 긴 기다림마침내 한국의 유령이 온다환영과도 같은 무대사라지지 않을 영원...,거부할수없는 강렬한 이끌림 가면속에 감춰진 아름다운 러브스토리 파리 오페라 하우...,거부 없다 강렬하다 림 가면 속 아름답다 러브스토리 파리 오페라 하우스 무명 무용수...,"['거부', '없다', '강렬하다', '림', '가면', '속', '아름답다', '...","b""\x9f\xf8\x93\xbb\x0b\x98\x00<6\xe5\x8a:\xb3t...",오페라,사랑,공포
3,190718,벤허,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20230902 20231119,2023-09-02,2023-09-01,LG아트센터 서울 LG SIGNATURE 홀,박은태신성록규현이지훈박민성서경수윤공주이정화최지혜이정열홍경수한지연류수화장혜린문은수김대...,8세이상,155분 인터미션 20분,대한민국 창작 뮤지컬의 압도적 수작의 귀환제2회 한국뮤지컬어워즈 대상 무대예술상 앙...,서기 26년 제정 로마의 박해에 신음하는 예루살렘 명망 높은 유대의 귀족 벤허는 로...,서기 제정 로마 박해 신음 예루살렘 명망 높다 유대 귀족 벤허 로마 장교 친구 메 ...,"['서기', '제정', '로마', '박해', '신음', '예루살렘', '명망', '...",b'(\xca\xab\xbb\xf5\xc3\xc3;*H\xa68\x1c\xd5\x8...,사랑,친구,NaN
4,190723,레미제라블 부산,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231011 20231119,2023-10-11,2023-10-01,부산 드림씨어터,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,만 7세이상,NaN,시대를 뛰어넘는 마스터피스10년 만의 위대한 귀환한국 라이선스 공연 10주년 기념역...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"['사랑', '용기', '희망', '대한', '서사시', '기구', '운명', '청...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...,사랑,친구,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,195267,페인터즈,httpticketimageinterparkcomPlayDictionaryDATAP...,퍼포먼스,20231101 20231231,2023-11-01,2023-11-01,명보아트홀,NaN,전체관람가,75분,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,사랑 수 없다 명의 페인터 그 환상 미술 공연 시즌 놀랍다 라이브 드로잉 감각 어 ...,"['사랑', '수', '없다', '명의', '페인터', '그', '환상', '미술'...",b'b1\xda\xbb\xaf\xcb\x03<\xfa\xdd\x82\xbaWw9;\...,사랑,친구,NaN
81,167486,장수탕 선녀님,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20210916 20240303,2021-09-16,2021-09-01,서울숲 씨어터 2관,한송이박시연한리예최승환최혜인박시온양다은이서은최성애김인섭이용호김명훈박나연변민슬권유진한...,12개월이상,60분,가족뮤지컬 연간 판매랭킹 1위 동네 낡은 목욕탕에서 만나는 신비로운 판타지의 세계...,겁먹지 마라 얘야 나는 저기 산속에 사는 선녀란다 우리 동네에는 오래된 목욕탕이...,겁 마르다 얘 나 저기 산속 선녀 란 우리 동네 오래되다 목욕탕 있다 큰길 새롭다 ...,"['겁', '마르다', '얘', '나', '저기', '산속', '선녀', '란', ...",b'\xc3}\xd0\xbb\x99\xe4\xc8;\xfe\xe3\x95\xb9\x...,웃음,사랑,NaN
82,169908,스크루테이프의 편지,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220101 20231230,2022-01-01,2022-01-01,북촌나래홀,NaN,만 12세이상,80분,나니아 연대기 작가 CS 루이스의 경험 사색 문학적 상상력으로 가득찬공연 뮤지컬...,경험 많고 노회한 고참 악마이자 악마특수공작대Devil Special Agency...,경험 많다 노회 고참 악마 이자 악마 특수 공작 의 보스 스크루 테이프 원수 밤낮 ...,"['경험', '많다', '노회', '고참', '악마', '이자', '악마', '특수...",b'\xf1\xc2\xab\xbb\xc2\xd40<\x88\xbc\xd5:\x98r...,사랑,NaN,NaN
83,173988,라면에 파송송,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220423 20240127,2022-04-23,2022-04-01,로즈아트홀 구노방극장,김재선김슬기김태경,만 10세이상,100분,작가 의도 인생은 맛있다 누군가는 현재의 삶이 달콤할 수도 있고 누군가는 현재...,비정규직을 전전하는 한 가정의 가장 다훈 우연히 보증금 백만원과 월세 십오만원에 나...,비정규직 전전

In [4]:
# 벡터를 문자열로 변환한 synopsis_vector 컬럼 값을 string_array에 저장
t_string_array = train_data.loc[0,"synopsis_vector"]
t_string_array

'b\'\\x0e\\x11u<\\xdeM1=\\xa1kX=\\xd11\\x06\\xbe?\\x03\\x8a\\xbc\\x13^u\\xbe\\xc1\\\\\\xa8>\\xdf\\x95\\xe3>S\\x1fp\\xbe\\xc8\\xa9\\x89\\xbe?P\\xf3=O\\x18]\\xbe\\xc9\\x9b\\x0b\\xbe\\x7fj\\x0b\\xbd(\\xe1\\xb1=\\x899\\xde\\xbd\\xa4\\x87\\x99\\xbd2\\xe3\\xb9\\xbd\\t\\xfc\\x03\\xbdg\\x1b\\xf5\\xbeB\\xf8>=\\xac\\x15\\x8d\\xbd\\x0e\\xf5\\xa2=\\xef \\x05\\xbe\\x8c\\x96\\x89\\xbe\\x1c\\xb8\\x1f\\xbe\\x9d\\x14\\xd7\\xbdq\\x94\\xb5\\xbd\\xab\\xc4\\x88=\\xf18b=^M`>\\xa0\\xf9\\x06>y"\\xf6\\xbd\\x90:\\xb8\\xbc\\x08W\\xcc=\\xe7H\\x9a>\\x08$\\xc7\\xbcY\\xbb\\t\\xbe\\xf06\\xa1>\\xd6{\\x87\\xbeY\\xe7~>\\xf4\\xcb:\\xbepH\\xe0=Q\\xf6(>\\x91\\x10\\x85>s\\xf1*=\\x06H\\x1e\\xbe\\x8bL\\x08=\\x15g\\x99=\\xe2\\xf9\\x06>\\xe0\\x19F>\\t\\x1c\\x80\\xbe6#\\xf8\\xbb\\xf8W\\xb8=J~\\xa1\\xbe\\xbe[y=\\x06\\x13F=\\xe9RO\\xbd\\x05Y\\xd1\\xbe\\xc2\\xd3\\xc9=7%$\\xbe\\xfe\\xffY\\xbd\\x86\\\\\\x9e=e\\xa6\\xec;\\xaay\\xa8\\xbe\\x86?\\xb8>\\xde\\xab\\\\=\\xe1\\x14\\xf9=~\\x9d,\\xbe\\x92\\x8c;\\xbd\\xb5E\\xac\\xbd \\x8eP>R\\xc

In [6]:
# 벡터를 문자열로 변환한 synopsis_vector 컬럼 값을 string_array에 저장
p_string_array = present_data.loc[0,"synopsis_vector"]
p_string_array

"b'\\r\\xc1\\x90\\xbb\\x04\\x8b\\xd4;\\x13\\xfd :\\xf2u\\x03;|\\xc6\\x88;`\\xbd\\x19\\xbc\\xf5\\x862:0!\\\\<\\xa4+\\x9d\\xbb.\\xd9G\\xbb0\\xa1\\xd0\\xbbU<\\x0b\\xbc\\xd7\\xdc\\x94\\xbb\\xd9G\\x1e;\\xbc\\xc2.:\\xe4(\\xc0\\xbb\\xb2:\\x9a:\\xd0\\xe3\\xaf\\xbb\\xb63]\\xba\\x1e+o\\xbc=\\x7f\\xc39\\x7f\\x17\\x0c;\\xa3\\xe0\\xbb9h\\xe2\\xa2\\xbboQ \\xbb0\\x03?;\\xe7\\xd8\\xa7\\xbb\\xc4\\xd0y\\xbb\\xe9\\x98\\x89\\xbb\\xa3k\\xcb:\\x9a\\x01\\x06<\\x97\\xda\\x82\\xba\\x802\\x9f:O>\\x9a\\xbb\\x9f0\\xaa\\xbbU\\x18\\x05<\\xe3-\\xcd\\xba#m\\xf5\\xbb\\xae\\x93\\xae\\xbbdpJ\\xbcPJ\\xbf:\\x99\\x1f\\xd8\\xbb\\n\\xafh\\xba\\x06\\x1f\\r\\xbb\\x89=\\x85;\\xb2s\\xcb\\xbb\\x87n\\x97\\xba\\xe9$\\x86\\xba\\xe6X@;\\x9d\\xa1\\xab;-\\xe7\\xb4;\\xab\\xa3\\x1b\\xbc\\x0bgt\\xb9F\\xd1\\xca:\\x8dO\\x02\\xbc\\xd3:\\x8f;d\\xaf\\xf3;6VP\\xba\\x83\\xba\\x11\\xbc\\xa3\\xb8\\xce\\xbaS6\\xa1;3\\xd5\\x1f:\\xe4\\xce_\\xba\\x13\\xa5\\xc1\\xbb!5K\\xbcan\\xce;M\\xfa\\x80;1\\xbf\\xa4;\\x9a%$\\xbc\\xbc/\\xba;&\\x0f\\xe3\\xbbp\\xcd\\

In [10]:
# string_array 를 다시 numpy 배열로 변환
t_numpy_array = np.fromstring(t_string_array,dtype="float32")
t_numpy_array

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_9964\1123377923.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  t_numpy_array = np.fromstring(t_string_array,dtype="float32")


array([1.78610014e+34, 1.78805880e+34, 1.49653414e-02, 6.74324873e+22,
       2.13011910e+17, 2.14440153e+26, 1.78679611e+34, 1.99501506e+17,
       2.05126951e+17, 8.96033764e-01, 4.10876595e-08, 2.12679607e+20,
       4.17763697e+21, 4.13204901e-08, 1.78857156e+34, 1.78806499e+34,
       2.47915185e+17, 5.37562737e-05, 1.83257571e+22, 2.36855441e-04,
       7.33029880e+22, 2.37838435e+17, 2.84965425e+29, 6.68421915e+22,
       5.42331109e-05, 2.15025095e-04, 1.75924448e-04, 6.68421915e+22,
       1.78739515e+34, 3.17426227e+09, 4.23121819e-05, 1.14536200e+21,
       4.58145026e+21, 2.36854787e-04, 9.03531583e-10, 1.14536235e+21,
       1.48034642e-05, 1.78822395e+34, 1.78796373e+34, 1.89592673e+17,
       1.99730548e+17, 4.33067977e-02, 1.75924448e-04, 1.83257515e+22,
       1.14536256e+21, 2.86340624e+20, 5.92136785e-05, 2.36854758e-04,
       1.14535947e+21, 1.78644899e+34, 1.78648080e+34, 1.78667058e+34,
       6.98450509e+31, 4.25142394e+21, 4.10876595e-08, 1.67105479e+22,
      

In [12]:
# string_array 를 다시 numpy 배열로 변환
p_numpy_array = np.fromstring(p_string_array,dtype="float32")
p_numpy_array

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_9964\2415368049.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  p_numpy_array = np.fromstring(p_string_array,dtype="float32")


array([4.36059604e+30, 3.31012640e-09, 6.74736489e-10, 1.04440924e+21,
       1.64350638e-07, 8.50718429e+20, 2.12779412e-07, 3.61411767e-09,
       2.93211916e+23, 2.09559838e+17, 2.26149208e+32, 4.10876595e-08,
       1.78878907e+34, 1.78657563e+34, 5.39623770e+19, 1.67105479e+22,
       1.69248728e-04, 4.17763697e+21, 8.58568228e-07, 2.74881950e-06,
       5.97081403e-19, 2.12075383e+17, 6.40841113e-13, 1.36852818e+22,
       1.04440924e+21, 1.83257391e+22, 1.78710918e+34, 1.98591076e+17,
       1.99712956e+17, 1.98600250e+17, 1.55570342e+13, 9.03529473e-10,
       1.14536235e+21, 1.83257988e+22, 1.83257492e+22, 2.36855398e-04,
       1.14535912e+21, 1.83257977e+22, 1.78710918e+34, 2.10988327e+17,
       2.56012349e+17, 4.05608602e-11, 7.33030105e+22, 1.78612613e+34,
       1.78638548e+34, 1.78796992e+34, 2.09638969e+17, 8.59163236e-04,
       8.31169578e-10, 5.34276836e-08, 2.66188174e+23, 1.04440924e+21,
       3.37466827e-06, 1.78780565e+34, 1.78793823e+34, 5.30442696e+28,
      

In [13]:
#모든행의 synopsis_vector 컬럼을 출력
train_data.loc[: ,"synopsis_vector"]

0        b'\x0e\x11u<\xdeM1=\xa1kX=\xd11\x06\xbe?\x03\x...
1        b'\xddC\xf5=I@Z=%\xfd\x8b\xbc\xd8\x03==\xd6q\x...
2        b"S\xa2\xa0<\xeb\xad\xd5<\xd6\x93\xbe\xbd\x01\...
3        b'^\xb5D>\xb63\xad=P\x8e >.\x97\x02\xbe\xee+\x...
4        b'@\x8a\xa8=\x8e\xd7\xe0=\x1e\x10\x9b=N%\xf6\x...
                               ...                        
20882    b'\xd5q4>\x9b\x12h\xbd*\xf4\x99>\xc0\xe2`>\xe1...
20883    b'\xd5q4>\x9b\x12h\xbd*\xf4\x99>\xc0\xe2`>\xe1...
20884    b'\xd5q4>\x9b\x12h\xbd*\xf4\x99>\xc0\xe2`>\xe1...
20885    b'\xd5q4>\x9b\x12h\xbd*\xf4\x99>\xc0\xe2`>\xe1...
20886    b'\xd5q4>\x9b\x12h\xbd*\xf4\x99>\xc0\xe2`>\xe1...
Name: synopsis_vector, Length: 20887, dtype: object

In [39]:
#모든행의 synopsis_vector 컬럼을 출력
p_string_array = present_data.loc[: ,"synopsis_vector"]
p_string_array

0     b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...
1     b'\xd8M\xd1\xbb[\x8d\x14<\x18\x1f\xf09v\x01\x0...
2     b"\x9f\xf8\x93\xbb\x0b\x98\x00<6\xe5\x8a:\xb3t...
3     b'(\xca\xab\xbb\xf5\xc3\xc3;*H\xa68\x1c\xd5\x8...
4     b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...
                            ...                        
80    b'b1\xda\xbb\xaf\xcb\x03<\xfa\xdd\x82\xbaWw9;\...
81    b'\xc3}\xd0\xbb\x99\xe4\xc8;\xfe\xe3\x95\xb9\x...
82    b'\xf1\xc2\xab\xbb\xc2\xd40<\x88\xbc\xd5:\x98r...
83    b"\x14d\x0b\xbc7\xeb\x1b<\xb4(8:SP\x05;)\xea\x...
84    b'mP\xed\xbb\xc7\xa7\x16<\xc2\xec\xe1:\xa3\x1e...
Name: synopsis_vector, Length: 85, dtype: object

In [44]:
##모든행의 synopsis_vector 컬럼을 numpy 배열로 변환해서 synopsis_numpy 컬럼에 대입
train_data.loc[: ,"synopsis_numpy"] = train_data.loc[: ,"synopsis_vector"].apply(lambda x: np.fromstring(x,dtype="float32") )

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_9964\1816109632.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  train_data.loc[: ,"synopsis_numpy"] = train_data.loc[: ,"synopsis_vector"].apply(lambda x: np.fromstring(x,dtype="float32") )


ValueError: string size must be a multiple of element size